In [1]:
from decimal import Decimal, getcontext
from fractions import Fraction
import math

In [171]:
class FrequencyTable:

    def __init__(self, freqs):
        self.counts = freqs
        self.total = sum(freqs.values())
        self.freqs = self.__generate_freqs()
        self.freq_ranges = self.__generate_freqs_range()

    def __generate_freqs(self, round=True):
        freqs = {symbol: Fraction(self.counts[symbol], self.total) for symbol in self.symbols}
        if not round:
            return freqs

        freqs = {symbol: self.floor_quantize(freqs[symbol]) for symbol in freqs}

        probs_sum = sum(freqs.values())
        for symbol, prob in freqs.items():
            if probs_sum >= 1:
                break
            if prob < self.floor_quantize(Fraction(1, 1) - probs_sum):
                freqs[symbol] *= 2
                probs_sum = sum(freqs.values())

        return freqs

    def __generate_freqs_range(self):
        freq_ranges = {}

        lower = Fraction(0, 1)
        for symbol, prob in self.freqs.items():
            freq_ranges[symbol] = (lower, prob)

            lower += prob

        return freq_ranges

    @property
    def symbols(self):
        return sorted(self.counts, key=lambda x: self.counts[x], reverse=True)

    @staticmethod
    def from_text(text):
        freqs = {}
        for symbol in set(text):
            freqs[symbol] = text.count(symbol)

        return FrequencyTable(freqs=freqs)

    @staticmethod
    def floor_quantize(number):
        floor_exponent = abs(math.floor(math.log2(number)))
        return Fraction(1, 2 ** floor_exponent)

In [166]:
from bisect import bisect_right
from tqdm import tqdm

class ArithmeticCoder:

    def encode(self, text):
        freqs = FrequencyTable.from_text(text)

        probs_sum = sum(freqs.freqs.values())
        lower, delta = Fraction(0, 1), Fraction(1, 1)
        for symbol in tqdm(text, desc='Encoding'):
            current_lower, current_delta = freqs.freq_ranges[symbol]
            lower += current_lower * delta
            delta *= current_delta

        return lower, len(text), freqs

    def decode(self, encoded, length, freqs):
        range_lower_bounds = [symbol_range[0] for symbol_range in freqs.freq_ranges.values()]

        decoded = ''
        for _ in range(length):
            symbol_idx = bisect_right(range_lower_bounds, encoded) - 1
            symbol = freqs.symbols[symbol_idx]
            decoded += symbol

            lower, delta = freqs.freq_ranges[symbol]
            encoded = (encoded - lower) / delta

        return decoded

In [30]:
with open('dickens.txt', 'rb') as file:
    data = file.read()

In [174]:
text = data[:10000].decode()

enc = ArithmeticCoder()
encoded, length, freqs = enc.encode(text)
decoded = enc.decode(encoded, len(text), freqs)

print(f'Encoded: {encoded}')
print(f'Decoded: {decoded}')
print(f'Successfully: {decoded == text}')

Encoding: 100%|██████████| 10000/10000 [00:09<00:00, 1036.78it/s]


Encoded: ...
Decoded: It was the best of times, it was the worst of times,
it was the age of wisdom, it was the age of foolishness,
it was the epoch of belief, it was the epoch of incredulity,
it was the season of Light, it was the season of Darkness,
it was the spring of hope, it was the winter of despair,
we had everything before us, we had nothing before us,
we were all going direct to Heaven, we were all going direct
the other way--in short, the period was so far like the present
period, that some of its noisiest authorities insisted on its
being received, for good or for evil, in the superlative degree
of comparison only.

There were a king with a large jaw and a queen with a plain face,
on the throne of England; there were a king with a large jaw and
a queen with a fair face, on the throne of France.  In both
countries it was clearer than crystal to the lords of the State
preserves of loaves and fishes, that things in general were
settled for ever.

It was the year of Our Lord on

In [18]:
import math

def normalize_number(fraction):
    precision = math.ceil(math.log2(fraction.numerator + 1) / 8)
    number = int((2 ** (precision * 8)) * fraction)

    return number, precision

In [175]:
x, precision = normalize_number(encoded)
math.log2(x) / 8

5575.997408150627

In [ ]:
class Serializer:

    def __init__(self, file_name):
        self.stream = open(file_name)